<a href="https://colab.research.google.com/github/RodyRuan/Kaggle_Popcorn/blob/main/Bag_of_Words_Meets_Bags_of_Popcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install gensim
!pip install nltk
!pip install scikit-learn
!pip install pandas


In [27]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import re
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# Carregar os dados
labeled_train_data = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
test_data = pd.read_csv('testData.tsv', delimiter='\t')


In [29]:
# Função para limpar e preprocessar os textos
def preprocess_text(text):
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in tokens if word not in stop_words]
    return filtered_words

# Aplicar preprocessamento nos textos
labeled_train_data['clean_review'] = labeled_train_data['review'].apply(preprocess_text)
test_data['clean_review'] = test_data['review'].apply(preprocess_text)


In [30]:
# Treinar o modelo Word2Vec
model = Word2Vec(sentences=labeled_train_data['clean_review'], vector_size=100, window=5, min_count=2, workers=4)
model.train(labeled_train_data['clean_review'], total_examples=len(labeled_train_data['clean_review']), epochs=10)


(28452612, 29878200)

In [31]:
def review_to_vector(review, model, num_features):
    feature_vec = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in review:
        if word in model.wv:
            num_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if num_words > 0:
        feature_vec = np.divide(feature_vec, num_words)
    return feature_vec

num_features = 100
train_data_vecs = np.array([review_to_vector(review, model, num_features) for review in labeled_train_data['clean_review']])
test_data_vecs = np.array([review_to_vector(review, model, num_features) for review in test_data['clean_review']])


In [32]:
# Dividir os dados de treino em treino e validação
X_train, X_valid, y_train, y_valid = train_test_split(train_data_vecs, labeled_train_data['sentiment'], test_size=0.2, random_state=42)

# Treinar o classificador RandomForest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Previsão e avaliação
y_pred = clf.predict(X_valid)
roc_score = roc_auc_score(y_valid, y_pred)
print(f'ROC AUC Score: {roc_score}')


ROC AUC Score: 0.8332186067067234


In [33]:
# Previsões no conjunto de teste
test_predictions = clf.predict(test_data_vecs)

# Criação do arquivo de submissão
submission = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predictions})
submission.to_csv('submission.csv', index=False)
